UK trade in services: service type by partner country, non-seasonally adjusted

In [1]:
from gssutils import *

scraper = Scraper('https://www.ons.gov.uk/businessindustryandtrade/' + \
                  'internationaltrade/datasets/uktradeinservicesservicetypebypartnercountrynonseasonallyadjusted')
scraper

## UK trade in services: service type by partner country, non-seasonally adjusted

Quarterly estimates of service type by partner country, non-seasonally adjusted.

### Distributions

1. UK trade in services: service type by partner country, non-seasonally adjusted ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/businessindustryandtrade/internationaltrade/datasets/uktradeinservicesservicetypebypartnercountrynonseasonallyadjusted/current/servicetypebycountry.xlsx))


In [2]:
tab = scraper.distribution().as_pandas(header=None)
tab.rename(columns=tab.iloc[0], inplace=True)
tab = tab.iloc[1:, :]
tab = tab.drop(tab.columns[[2,4,8]], axis = 1)
tab

,Direction,Service type code,Country code,2016,2017,2018,2016Q1,2016Q2,2016Q3,2016Q4,2017Q1,2017Q2,2017Q3,2017Q4,2018Q1,2018Q2,2018Q3,2018Q4,2019Q1
1,Exports,0,AT,1035,1192,1236,228,254,278,275,278,303,298,313,270,322,325,319,279
2,Exports,0,BE,4253,5234,4894,980,1040,1082,1151,1228,1421,1445,1140,1103,1259,1220,1312,1160
3,Exports,0,BG,457,498,558,94,114,129,120,101,128,140,129,112,149,150,147,124
4,Exports,0,HR,196,195,202,41,48,52,55,36,47,58,54,41,58,50,53,47
5,Exports,0,CY,807,812,860,174,202,219,212,170,194,225,223,166,236,223,235,215
6,Exports,0,CZ,1004,1049,1048,214,250,268,272,235,275,276,263,260,252,269,267,240
7,Exports,0,DK,3570,3617,4029,796,884,905,985,806,928,917,966,914,1031,1024,1060,862
8,Exports,0,EE,67,80,87,13,17,18,19,20,20,21,19,14,23,25,25,15
9,Exports,0,FI,1203,1249,1230,280,302,294,327,293,308,321,327,288,309,305,328,387
10,Exports,0,FR,15723,16770,17596,3578,3917,3997,4231,3872,4201,4388,4309,4110,4476,4383,4627,4316


In [3]:
tab.columns.values[0] = 'Flow'
tab.columns.values[1] = 'Pink Book Services'
tab.columns.values[2] = 'ONS Partner Geography'

In [4]:
new_table = pd.melt(tab, id_vars=['Flow','Pink Book Services','ONS Partner Geography'], var_name='Period', value_name='Value')

In [5]:
new_table['Period'] = new_table['Period'].astype(str)

In [6]:
import re
YEAR_RE = re.compile(r'[0-9]{4}')
YEAR_MONTH_RE = re.compile(r'([0-9]{4})\s+(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)')
YEAR_QUARTER_RE = re.compile(r'([0-9]{4})(Q[1-4])')

class Re(object):
  def __init__(self):
    self.last_match = None
  def fullmatch(self,pattern,text):
    self.last_match = re.fullmatch(pattern,text)
    return self.last_match

def time2period(t):
    gre = Re()
    if gre.fullmatch(YEAR_RE, t):
        return f"year/{t}"
    elif gre.fullmatch(YEAR_MONTH_RE, t):
        year, month = gre.last_match.groups()
        month_num = {'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04', 'MAY': '05', 'JUN': '06',
                     'JUL': '07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'}.get(month)
        return f"month/{year}-{month_num}"
    elif gre.fullmatch(YEAR_QUARTER_RE, t):
        year, quarter = gre.last_match.groups()
        return f"quarter/{year}-{quarter}"
    else:
        print(f"no match for {t}")

new_table['Period'] = new_table['Period'].apply(time2period)

In [7]:
new_table['Flow'] = new_table['Flow'].map(lambda s: s.lower().strip())

In [8]:
new_table['Seasonal Adjustment'] =  'NSA'
new_table['Measure Type'] =  'GBP Total'
new_table['Unit'] =  'gbp-million'

In [9]:
def user_perc(x):
    
    if (str(x) ==  '-') : 
        return 'itis-nil'
    elif ((str(x) == '..')):
        return 'disclosive'
    else:
        return None
    
new_table['Marker'] = new_table.apply(lambda row: user_perc(row['Value']), axis = 1)

In [10]:
new_table['Value'] = pd.to_numeric(new_table['Value'], errors = 'coerce')

In [11]:
new_table = new_table[['ONS Partner Geography', 'Period','Flow','Pink Book Services', 'Seasonal Adjustment', 'Measure Type','Value','Unit','Marker' ]]